<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

# 1. Building Dataframe of Postal Codes, Boroughs, and Neighborhoods

In [1]:
# loading libraries

!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 


import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [2]:
pip install jupyterlab

     |████████████████████████████████| 6.4MB 6.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 7.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.8MB 7.1MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install geocoder

     |████████████████████████████████| 102kB 7.9MB/s ta 0:00:011
     |████████████████████████████████| 829kB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 81kB 19.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install geopandas

     |████████████████████████████████| 921kB 9.3MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 32.9MB/s eta 0:00:01
     |████████████████████████████████| 11.8MB 32.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import geocoder # import geocoder

print ('imported')

imported


In [9]:
from geopy.geocoders import Nominatim # import nominatim

print ('imported')

imported


In [10]:
# webscraping data

from bs4 import BeautifulSoup
import requests

with open('List of postal codes of Canada_ M - Wikipedia.html') as html_file: 
    soup = BeautifulSoup(html_file, 'lxml')
    
print(soup.prettify())

<!DOCTYPE html>
<!-- saved from url=(0063)https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M -->
<html class="client-js ve-not-available" dir="ltr" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XhnDpwpAAEAAAChwLdcAAAEL","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":933624196,"wgRevis

In [11]:
# sorting data into lists

table = soup.table

postalcode = list()
borough = list()
neighborhood = list()

for row in soup.find_all('table')[0].tbody.find_all('tr')[0:]:
    
    postalcode.append(row.find_all('td')[0].text)
    
print(postalcode)


for row in soup.find_all('table')[0].tbody.find_all('tr')[0:]:
    
    borough.append(row.find_all('td')[1].text)
    
print(borough)


for row in soup.find_all('table')[0].tbody.find_all('tr')[0:]:
    
    neighborhood.append(row.find_all('td')[2].text)


print(neighborhood)

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M6A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B', 'M1B', 'M2B', 'M3B', 'M4B', 'M4B', 'M5B', 'M5B', 'M6B', 'M7B', 'M8B', 'M9B', 'M9B', 'M9B', 'M9B', 'M9B', 'M1C', 'M1C', 'M1C', 'M2C', 'M3C', 'M3C', 'M4C', 'M5C', 'M6C', 'M7C', 'M8C', 'M9C', 'M9C', 'M9C', 'M9C', 'M1E', 'M1E', 'M1E', 'M2E', 'M3E', 'M4E', 'M5E', 'M6E', 'M7E', 'M8E', 'M9E', 'M1G', 'M2G', 'M3G', 'M4G', 'M5G', 'M6G', 'M7G', 'M8G', 'M9G', 'M1H', 'M2H', 'M3H', 'M3H', 'M3H', 'M4H', 'M5H', 'M5H', 'M5H', 'M6H', 'M6H', 'M7H', 'M8H', 'M9H', 'M1J', 'M2J', 'M2J', 'M2J', 'M3J', 'M3J', 'M4J', 'M5J', 'M5J', 'M5J', 'M6J', 'M6J', 'M7J', 'M8J', 'M9J', 'M1K', 'M1K', 'M1K', 'M2K', 'M3K', 'M3K', 'M4K', 'M4K', 'M5K', 'M5K', 'M6K', 'M6K', 'M6K', 'M7K', 'M8K', 'M9K', 'M1L', 'M1L', 'M1L', 'M2L', 'M2L', 'M3L', 'M4L', 'M4L', 'M5L', 'M5L', 'M6L', 'M6L', 'M6L', 'M7L', 'M8L', 'M9L', 'M1M', 'M1M', 'M1M', 'M2M', 'M2M', 'M3M', 'M4M', 'M5M', 'M5M', 'M6M', 'M6M', 'M6M', 'M6M', 'M7M', 'M8M', 'M9M', 'M9M', 'M1N', 'M1N', 'M2N', 'M3N'

In [12]:
# cleaning data and building dataframe

neighborhood = [x.replace('\n', '') for x in neighborhood]

df = pd.DataFrame({'PostalCode': postalcode, 'Borough': borough, 'Neighborhood': neighborhood})
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [13]:
# removing 'not assigned' rows from borough column

df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [14]:
# converting 'not assigned' values in neighborhood column into corresponding values from borough column

df.Neighborhood[df.Neighborhood == 'Not assigned'] = df.Borough
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [15]:
# building second dataframe to group rows by postal code and aggregate neighborhood values

df2 = df.groupby('PostalCode')['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index()

df2.head()

,PostalCode,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [16]:
# merging both dataframes to insert borough column into the second dataframe and renaming resulting dataframe 

result = pd.merge(df2,
                 df[['PostalCode', 'Borough']],
                 on='PostalCode',
                 how='left')

df = result
df.head()

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
4,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough


In [17]:
# cleaning data

df.drop_duplicates(inplace=True)
df.head()

,PostalCode,Neighborhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
2,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
5,M1E,"Guildwood, Morningside, West Hill",Scarborough
8,M1G,Woburn,Scarborough
9,M1H,Cedarbrae,Scarborough


In [18]:
# organizing columns

cols = ['PostalCode', 'Borough', 'Neighborhood']
df = df[cols]
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
5,M1E,Scarborough,"Guildwood, Morningside, West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


In [19]:
# sorting data

df.sort_values(['Borough'], inplace=True)
df.reset_index(drop= True, inplace=True)

In [20]:
# showing completed dataframe with postal code, borough, and neighborhood values

df

,PostalCode,Borough,Neighborhood
0,M4S,Central Toronto,Davisville
1,M5N,Central Toronto,Roselawn
2,M4R,Central Toronto,North Toronto West
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West"
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
5,M4T,Central Toronto,"Moore Park, Summerhill East"
6,M4N,Central Toronto,Lawrence Park
7,M4P,Central Toronto,Davisville North
8,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
9,M4W,Downtown Toronto,Rosedale


In [21]:
# verifying size of dataframe

df.shape

(103, 3)

#  2. Building Dataframe of Latitude and Longitude Coordinates

In [22]:
# geocoding for latitude and longitude from postal code using the arcgis geocoder

Lat_list=[]
Lng_list=[]
for i in range(df.shape[0]):
    address='{}, Toronto, Ontario'.format(df.at[i,'PostalCode'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])
    
print('Lat list:', Lat_list, "\n") 
print('Lng list:', Lng_list)

Lat list: [43.70339500000006, 43.711941154000044, 43.71452278400005, 43.69478500000008, 43.674840000000074, 43.69068500000003, 43.72842000000003, 43.71275500000007, 43.68607377100005, 43.68220500000007, 43.666585000000055, 43.65029500000003, 43.65736301100003, 43.64516015600003, 43.65609081300005, 43.64970000000005, 43.64710000000008, 43.64839500000005, 43.65121000000005, 43.66816000000006, 43.653530000000046, 43.663110000000074, 43.66115033500006, 43.66878132800008, 43.630210000000034, 43.648690000000045, 43.64081500000003, 43.64828000000006, 43.648690000000045, 43.660628993000046, 43.66796500000004, 43.68317787500007, 43.67653121600006, 43.688765344000046, 43.70124000000004, 43.70949500000006, 43.689640000000054, 43.707535000000064, 43.648573449000025, 43.624630000000025, 43.74314500000003, 43.63283500000006, 43.653760000000034, 43.60113082100003, 43.61220000000003, 43.69650500000006, 43.68681000000004, 43.65034698100004, 43.71174000000008, 43.648690000000045, 43.71381000000008, 43.7

In [23]:
# showing completed dataframe with latitude and longitude values

df['Latitude'] = Lat_list
df['Longitude'] = Lng_list

df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4S,Central Toronto,Davisville,43.703395,-79.385964
1,M5N,Central Toronto,Roselawn,43.711941,-79.419120
2,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.694785,-79.414405
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.674840,-79.403768
5,M4T,Central Toronto,"Moore Park, Summerhill East",43.690685,-79.382946
6,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
7,M4P,Central Toronto,Davisville North,43.712755,-79.388514
8,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686074,-79.402265
9,M4W,Downtown Toronto,Rosedale,43.682205,-79.377945


In [24]:
# verifying size of dataframe

df.shape

(103, 5)

# 3. Building Map and Clustering Neighborhoods by Top Venue Categories

In [25]:
# importing map rendering library

import folium 

In [26]:
# checking toronto geographical coordinates

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [28]:
# entering foursquare credentials

CLIENT_ID = 'MSWQU2E54PNNZER5WFZQ4P3YMHKXXESTOBXLSPUFRDASCJKI' # your Foursquare ID
CLIENT_SECRET = '5VTGKWKXGLXY4KSLLXVMEWIMQZ0BIVAEEGFL2BXXWZBRRQJI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MSWQU2E54PNNZER5WFZQ4P3YMHKXXESTOBXLSPUFRDASCJKI
CLIENT_SECRET:5VTGKWKXGLXY4KSLLXVMEWIMQZ0BIVAEEGFL2BXXWZBRRQJI


In [29]:
# setting request limit for url

LIMIT = 100

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
# showing venue neighborhoods

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Davisville
Roselawn
North Toronto West
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Moore Park, Summerhill East
Lawrence Park
Davisville North
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Church and Wellesley
Harbourfront
Ryerson, Garden District
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
St. James Town
Cabbagetown, St. James Town
Chinatown, Grange Park, Kensington Market
Harbord, University of Toronto
Queen's Park
Christie
Harbourfront East, Toronto Islands, Union Station
Stn A PO Boxes 25 The Esplanade
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
First Canadian Place, Underground city
Business Reply Mail Processing Centre 969 Eastern
Studio District
The Beaches West, India Bazaar
The Danforth West, Riverdale
The Beaches
East Toronto
Thorncliffe Park
Leaside
Woodbine Heights
Woodbine 

In [34]:
# understanding dataframe size and composition

print (toronto_venues.shape)
toronto_venues.head(20)

(2451, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville,43.703395,-79.385964,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
1,Davisville,43.703395,-79.385964,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
2,Davisville,43.703395,-79.385964,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
3,Davisville,43.703395,-79.385964,GoodLife Fitness,43.700802,-79.386417,Gym
4,Davisville,43.703395,-79.385964,June Rowlands Park,43.700517,-79.389189,Park
5,Davisville,43.703395,-79.385964,XO Gelato,43.705177,-79.388793,Dessert Shop
6,Davisville,43.703395,-79.385964,Zee Grill,43.704985,-79.388476,Seafood Restaurant
7,Davisville,43.703395,-79.385964,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant
8,Davisville,43.703395,-79.385964,Bread and Butter,43.701582,-79.387359,Sandwich Place
9,Davisville,43.703395,-79.385964,Viva Napoli,43.705752,-79.389125,Pizza Place


In [35]:
# sorting venues by neighborhood and count

toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,12,12,12,12,12,12
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",15,15,15,15,15,15
"Alderwood, Long Branch",4,4,4,4,4,4


In [36]:
# verifying unique categories

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 259 uniques categories.


In [37]:
# organizing venues by category

# one hot encoding
t_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]

t_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [38]:
# verifying new dataframe size

t_onehot.shape

(2451, 259)

In [39]:
# sorting new dataframe

tg = t_onehot.groupby('Neighborhood').mean().reset_index()
tg.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.03,0.0,0.01,0.0,0.03,0.00,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.03,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.03,0.0,0.0,0.000000,0.0,0.01,0.03,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.070000,0.0,0.

In [40]:
# verifying new dataframe size

tg.shape

(97, 259)

In [41]:
# gathering and sorting all top venues by neighborhood

num_top_venues = 5

for hood in tg['Neighborhood']:
    print("----"+hood+"----")
    temp = tg[tg['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4   Restaurant  0.03


----Agincourt----
                   venue  freq
0          Shopping Mall  0.17
1     Chinese Restaurant  0.08
2    Shanghai Restaurant  0.08
3                   Pool  0.08
4  Vietnamese Restaurant  0.08


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
           venue  freq
0       Pharmacy   1.0
1    Yoga Studio   0.0
2      Nightclub   0.0
3    Men's Store   0.0
4  Metro Station   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.13
1                  Park  0.07
2        Hardware Store  0.07
3            Beer Store  0.07
4  Caribbean Restaurant  0.07


----Alderwood, Long Branch----
                venue  freq
0   Convenience Store  0.25
1                 Pub  0.25
2            

In [42]:
# defining function to gather top venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tg['Neighborhood']

for ind in np.arange(tg.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tg.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Gastropub
1,Agincourt,Shopping Mall,Bubble Tea Shop,Sushi Restaurant,Vietnamese Restaurant,Skating Rink
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Women's Store,Donut Shop,Flea Market,Fish Market
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Hardware Store,Japanese Restaurant,Pizza Place,Park
4,"Alderwood, Long Branch",Convenience Store,Pub,Gym,Athletics & Sports,Women's Store


In [44]:
# set number of clusters
kclusters = 2

t_clustering = tg.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(t_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int32)

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tdf = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tdf = tdf.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tdf.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4S,Central Toronto,Davisville,43.703395,-79.385964,1.0,Dessert Shop,Pizza Place,Sandwich Place,Café,Italian Restaurant
1,M5N,Central Toronto,Roselawn,43.711941,-79.419120,NaN,NaN,NaN,NaN,NaN,NaN
2,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0.0,Playground,Garden,Gym Pool,Park,Event Space
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.694785,-79.414405,0.0,Home Service,Park,Women's Store,Farm,Eastern European Restaurant
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.674840,-79.403768,1.0,Sandwich Place,Café,Cosmetics Shop,Coffee Shop,Liquor Store


In [46]:
# cleaning data

tdf = tdf.dropna()

In [47]:
# cleaning data

tdf = tdf.astype({"Cluster Labels": int})
tdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4S,Central Toronto,Davisville,43.703395,-79.385964,1,Dessert Shop,Pizza Place,Sandwich Place,Café,Italian Restaurant
2,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0,Playground,Garden,Gym Pool,Park,Event Space
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.694785,-79.414405,0,Home Service,Park,Women's Store,Farm,Eastern European Restaurant
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.674840,-79.403768,1,Sandwich Place,Café,Cosmetics Shop,Coffee Shop,Liquor Store
5,M4T,Central Toronto,"Moore Park, Summerhill East",43.690685,-79.382946,1,Convenience Store,Tennis Court,Gym,Trail,Women's Store


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [49]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tdf['Latitude'], tdf['Longitude'], tdf['Neighborhood'], tdf['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
# understanding clusters

In [51]:
red_clusters = tdf.loc[tdf['Cluster Labels'] == 0, tdf.columns[[6]]]

In [52]:
blue_clusters = tdf.loc[tdf['Cluster Labels'] == 1, tdf.columns[[6]]]

In [53]:
count = red_clusters['1st Most Common Venue'].value_counts()
print(count.head())

Park              8
Playground        2
Dog Run           1
Discount Store    1
Bakery            1
Name: 1st Most Common Venue, dtype: int64


In [54]:
count = blue_clusters['1st Most Common Venue'].value_counts()
print(count.head())

Coffee Shop          21
Pizza Place           8
Convenience Store     5
Café                  4
Playground            3
Name: 1st Most Common Venue, dtype: int64


# Data Insights

So if you were interested in moving to or around Toronto but wanted to understand the top venues trending in different neighborhoods then what this data can tell us is:

If you're interested in living in a place where the top venues tended to be parks, playgrounds, dog runs, discount stores, and bakeries, then the red clusters may be ideal neighborhoods for you.

If you're interested in living in a place where the top venues tended to be coffee shops, pizza places, convenience stores, cafes, and playgrounds, then the blue clusters may be ideal neighborhoods for you.